# Лабораторная работа 1. Введение в машинное обучение. Обучение с учителем. Задача регрессии

<b>Традиционное предупреждение для всех лабораторных работ:</b> перед обучением моделей необходимо выполнить предварительную обработку данных, которая <b>обязательно</b> включает в себя:
- заполнение пропущенных значений (рекомедуется логика заполнения пропусков на основе типа данных, которая использовалась в РГР по Практикуму);
- преобразование категориальных признаков в числовые (используйте one-hot кодирование или map; используйте знания с Практикума).

Предобработка может включать в себя другие действия, но выполнение описанных выше действий обязательно.

Сделайте это один раз и сохраните в отдельный csv файл, а потом его используйте.

<b>Выполните следующие задания:</b>
- загрузите датасет для регрессии, выделите целевой признак и предикторы, разбейте данные на обучающую и тестовую выборку;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (линейная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (полиномиальная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- вычислите значения метрик $R^2$, MAE, MSE, RMSE, MAPE для всех обученных моделей; выберите лучшую модель;
- самостоятельно реализуйте (желательно в виде класса) модель линейной регрессии с регуляризацией (можете выбрать L1 или L2);
- самостоятельно реализуйте вычисление всех используемых метрик (в виде функций, принимающих два аргумента);
- обучите вашу модель линейной регрессии на ваших данных; оцените качество с помощью реализованных вами метрик.

In [62]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from math import sqrt
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

Функция для вычисления метрик

In [63]:
def metrics(test, predict):
    print(f'MAE: {mean_absolute_error(test, predict)}')
    print(f'MSE: {mean_squared_error(test, predict)}')
    print(f'RMSE: {sqrt(mean_squared_error(test, predict))}')
    print(f'MAPE: {sqrt(mean_absolute_percentage_error(test, predict))}')
    print(f'R^2: {r2_score(test, predict)}')

Функция для заполнения пропусков в выборке

In [64]:
def fill_empty_cell(column_name, df):
    if df.dtypes[column_name] == "float64":
        df[column_name] = df[column_name].fillna(df[column_name].mean())
    elif df.dtypes[column_name] == "int64":
        df[column_name] = df[column_name].fillna(df[column_name].median())
    else:
        df[column_name] = df[column_name].fillna(df[column_name].mode())

Ridge

In [65]:
def create_ridge_with_hyperparameter(X_train, y_train) -> Ridge:
    alpha = {'alpha': np.arange(0.1, 1, 0.1)}
    ridge = GridSearchCV(Ridge(), alpha).fit(X_train, y_train)
    return ridge

Lasso

In [66]:
def create_lasso_with_hyperparameter(X_train, y_train) -> Lasso:
    alpha = {'alpha': np.arange(0.2, 1, 0.2)}
    lasso = RandomizedSearchCV(Lasso(), alpha).fit(X_train, y_train)
    return lasso

In [85]:
table = pd.read_csv("../data/energy_task.csv")
# table.drop(["date"], axis=1, inplace=True)
table
# table.head(5)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,11-01-2016 17:00,60,30,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,...,18.2000,48.900000,17.033333,45.5300,6.60,733.5,92.000000,7.000000,63.000000,5.3
1,11-01-2016 17:10,60,30,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,...,18.2000,48.863333,17.066667,45.5600,6.48,733.6,92.000000,6.666667,59.166667,5.2
2,11-01-2016 17:20,50,30,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,...,18.2000,48.730000,17.000000,45.5000,6.37,733.7,92.000000,6.333333,55.333333,5.1
3,11-01-2016 17:30,50,40,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,...,18.1000,48.590000,17.000000,45.4000,6.25,733.8,92.000000,6.000000,51.500000,5.0
4,11-01-2016 17:40,60,40,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,...,18.1000,48.590000,17.000000,45.4000,6.13,733.9,92.000000,5.666667,47.666667,4.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,27-05-2016 17:20,100,0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,...,24.7000,50.074000,23.200000,46.7900,22.70,755.2,55.666667,3.333333,23.666667,13.3
19731,27-05-2016 17:30,90,0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,...,24.7000,49.790000,23.200000,46.7900,22.60,755.2,56.000000,3.500000,24.500000,13.3
19732,27-05-2016 17:40,270,10,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,...,24.7000,49.660000,23.200000,46.7900,22.50,755.2,56.333333,3.666667,25.333333,13.3
19733,27-05-2016 17:50,420,10,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,...,24.6625,49.518750,23.200000,46.8175,22.30,755.2,56.666667,3.833333,26.166667,13.2


In [96]:
table['date'] = table['date'].apply(lambda x: x.split('-')[1])

TypeError: unsupported operand type(s) for +: 'float' and 'str'

Проверка датасета на наличие пустых данных

In [68]:
null_columns = []
for i in table.columns:
    if len(table[table[i].isnull()]) > 0:
        print(f"В столбце {i} есть пустые элементы")
        null_columns.append(i)
print(null_columns)

В столбце RH_2 есть пустые элементы
В столбце RH_4 есть пустые элементы
В столбце RH_5 есть пустые элементы
В столбце RH_6 есть пустые элементы
В столбце T7 есть пустые элементы
В столбце T8 есть пустые элементы
В столбце T9 есть пустые элементы
В столбце RH_9 есть пустые элементы
В столбце T_out есть пустые элементы
В столбце Press_mm_hg есть пустые элементы
В столбце RH_out есть пустые элементы
В столбце Visibility есть пустые элементы
['RH_2', 'RH_4', 'RH_5', 'RH_6', 'T7', 'T8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Visibility']


In [69]:
for name in null_columns:
    fill_empty_cell(name, table)

In [70]:
for i in table.columns:
    if len(table[table[i].isnull()]) > 0:
        print(f"В столбце {i} есть пустые элементы")

In [71]:
table.to_csv('../data/energy_task_compleated.csv') 

Создание обучающей выборки

In [72]:
y = table["Appliances"]
X = table.drop(["Appliances"], axis=1)

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

1. Простая линейная регрессия

In [74]:
lr = LinearRegression().fit(X_train, y_train)
y1_predict = lr.predict(X_test)
metrics(y_test, y1_predict)

MAE: 52.96789474656608
MSE: 9037.565781570385
RMSE: 95.06611268780472
MAPE: 0.7823947144754213
R^2: 0.16702458909428375


In [75]:
from pathlib import Path
import os
import sys

sys.path.append(str(Path(os.getcwd()).parent))

In [76]:
from mylib.Metrics import all_metrics


all_metrics(y_test, y1_predict)

MAE: 52.96789474656608
MSE: 9037.565781570385
RMSE: 95.06611268780472
MAPE: 0.612141489239076
R^2: 0.16702458909428375


2. L1 - Lasso (подбор гиперпараметра + модель)

In [77]:
l = create_lasso_with_hyperparameter(X_train, y_train)
y2_predict = l.predict(X_test)
metrics(y_test, y2_predict)

c:\Users\bugae\ML\venv\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


MAE: 52.90925274815981
MSE: 9044.870333685314
RMSE: 95.10452320308069
MAPE: 0.7817002016370093
R^2: 0.16635134228795656


3. L2 - Ridge (подбор гиперпараметра + модель)

In [78]:
r = create_ridge_with_hyperparameter(X_train, y_train)
y3_predict = r.predict(X_test)
metrics(y_test, y3_predict)

MAE: 52.96771597643805
MSE: 9037.59341153058
RMSE: 95.06625800740545
MAPE: 0.782392801024203
R^2: 0.16702204249290853


4. Полиномиальная регрессия

In [79]:
table_p = table.drop(columns=["T1", "RH_4", "RH_6", "T7"])
y_p = table_p["Appliances"]
X_p = table_p.drop(["Appliances"], axis=1)
X_p_train, X_p_test, y_p_train, y_p_test = train_test_split(X_p, y_p, test_size=0.5, random_state=42)

In [80]:
p = PolynomialFeatures(2)
X_PolynomialFeatures_train = p.fit_transform(X_p_train)
X_PolynomialFeatures_test = p.fit_transform(X_p_test)

In [81]:
lr2 = LinearRegression().fit(X_PolynomialFeatures_train, y_p_train)
y_PolynomialFeatures_predict = lr2.predict(X_PolynomialFeatures_test)
metrics(y_p_test, y_PolynomialFeatures_predict)

MAE: 51.22504792139761
MSE: 8379.987073571694
RMSE: 91.54226932718947
MAPE: 0.7570103636878832
R^2: 0.2214063427791635


5. Полиномиальная регрессия + Ridge

In [82]:
r_p = create_ridge_with_hyperparameter(X_PolynomialFeatures_train, y_p_train)
y_p_ridge_predict = r_p.predict(X_PolynomialFeatures_test)
metrics(y_p_test, y_p_ridge_predict)

MAE: 51.08247223497446
MSE: 8368.374996580362
RMSE: 91.47882266721824
MAPE: 0.75525861732351
R^2: 0.22248523340432025


6. Полиномиальная регрессия + Lasso

In [83]:
l_p = create_lasso_with_hyperparameter(X_PolynomialFeatures_train, y_p_train)
y_p_lasso_predict = l_p.predict(X_PolynomialFeatures_test)
metrics(y_p_test, y_p_ridge_predict)

c:\Users\bugae\ML\venv\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\bugae\ML\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.095e+07, tolerance: 8.158e+03
  model = cd_fast.enet_coordinate_descent(
c:\Users\bugae\ML\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.964e+07, tolerance: 7.867e+03
  model = cd_fast.enet_coordinate_descent(
c:\Users\bugae\ML\venv\lib\site-packages\sklearn\linear_model\_co

KeyboardInterrupt: 